In [1]:
#start rafiki
!bash ./scripts/start.sh

---------------------------------------
| Creating Docker swarm for Rafiki... |
---------------------------------------
Error response from daemon: This node is already part of a swarm. Use "docker swarm leave" to leave this swarm and join another one.
Failed to init Docker swarm - continuing...
Error response from daemon: network with name rafiki already exists
Failed to create Docker network for swarm - continuing...
------------------------------------
| Creating Rafiki's logs folder... |
------------------------------------
Created Rafiki's logs directory!
------------------------------------------------
| Pulling images for Rafiki from Docker Hub... |
------------------------------------------------
Pulling images required by Rafiki from Docker Hub...
postgres:10.5 already exists locally
redis:5.0-rc already exists locally
rafikiai/rafiki_admin:0.0.4 already exists locally
rafikiai/rafiki_advisor:0.0.4 already exists locally
rafikiai/rafiki_worker:0.0.4 already exists locally
rafi

In [30]:
#stop rafiki
!bash ./scripts/stop.sh

---------------------------
| Stopping Rafiki's DB... |
---------------------------
rafiki_db
------------------------------
| Stopping Rafiki's Cache... |
------------------------------
rafiki_cache
------------------------------
| Stopping Rafiki's Admin... |
------------------------------
rafiki_admin
--------------------------------
| Stopping Rafiki's Advisor... |
--------------------------------
rafiki_advisor
You'll need to destroy your machine's Docker swarm & Rafiki's logs folder at /home/wubiao/log/rafiki manually


In [70]:
!cat .env.sh

export POSTGRES_HOST=rafiki_db
export POSTGRES_PORT=5432
export POSTGRES_USER=rafiki
export POSTGRES_DB=rafiki
export POSTGRES_PASSWORD=rafiki
export DOCKER_NETWORK=rafiki
#export LOGS_FOLDER_PATH=/var/log/rafiki
export LOGS_FOLDER_PATH=/home/wubiao/log/rafiki
export ADMIN_HOST=rafiki_admin
export ADMIN_PORT=8000
export ADVISOR_HOST=rafiki_advisor
export ADVISOR_PORT=8001
export REDIS_HOST=rafiki_cache
export REDIS_PORT=6379
export PREDICTOR_PORT=8002
export RAFIKI_IP_ADDRESS=127.0.0.1
export RAFIKI_IMAGE_ADMIN=rafikiai/rafiki_admin
export RAFIKI_IMAGE_ADVISOR=rafikiai/rafiki_advisor
export RAFIKI_IMAGE_WORKER=rafikiai/rafiki_worker
export RAFIKI_IMAGE_PREDICTOR=rafikiai/rafiki_predictor
export IMAGE_POSTGRES=postgres:10.5
export IMAGE_REDIS=redis:5.0-rc
export RAFIKI_VERSION=0.0.4
export PYTHONPATH=$PWD


In [2]:
#start client
from rafiki.client import Client
client = Client(admin_host='localhost', admin_port=8000)
client.login(email='superadmin@rafiki', password='rafiki')

{'user_id': '71f6705c-ea09-4d98-98d1-25181dd34ea2', 'user_type': 'SUPERADMIN'}

#add models for default IMAGE_CLASSIFICATION
client.create_model(
    name='TfSingleHiddenLayer',
    task='IMAGE_CLASSIFICATION',
    model_file_path='examples/models/image_classification/TfSingleHiddenLayer.py',
    model_class='TfSingleHiddenLayer'
)

client.create_model(
    name='SkDt',
    task='IMAGE_CLASSIFICATION',
    model_file_path='examples/models/image_classification/SkDt.py',
    model_class='SkDt'
)

#check loaded models by task
client.get_models_of_task(task='IMAGE_CLASSIFICATION')

In [3]:
#add model for FEATURE_VECTOR_CLASSIFICATION
client.create_model(
    name='SkDtVector',
    task='FEATURE_VECTOR_CLASSIFICATION',
    model_file_path='examples/models/feature_vector_classification/SkDt.py',
    model_class='SkDtVector'
)

client.create_model(
    name='SkSvmVector',
    task='FEATURE_VECTOR_CLASSIFICATION',
    model_file_path='examples/models/feature_vector_classification/SkSvm.py',
    model_class='SkSvmVector'
)

client.create_model(
    name='TfSingleHiddenLayerVector',
    task='FEATURE_VECTOR_CLASSIFICATION',
    model_file_path='examples/models/feature_vector_classification/TfSingleHiddenLayer.py',
    model_class='TfSingleHiddenLayerVector'
)


#check loaded models by task
client.get_models_of_task(task='FEATURE_VECTOR_CLASSIFICATION')

[{'datetime_created': 'Sun, 28 Oct 2018 12:02:19 GMT',
  'docker_image': 'rafikiai/rafiki_worker:0.0.4',
  'model_class': 'SkDtVector',
  'name': 'SkDtVector',
  'task': 'FEATURE_VECTOR_CLASSIFICATION',
  'user_id': '71f6705c-ea09-4d98-98d1-25181dd34ea2'},
 {'datetime_created': 'Sun, 28 Oct 2018 12:02:19 GMT',
  'docker_image': 'rafikiai/rafiki_worker:0.0.4',
  'model_class': 'SkSvmVector',
  'name': 'SkSvmVector',
  'task': 'FEATURE_VECTOR_CLASSIFICATION',
  'user_id': '71f6705c-ea09-4d98-98d1-25181dd34ea2'},
 {'datetime_created': 'Sun, 28 Oct 2018 12:02:19 GMT',
  'docker_image': 'rafikiai/rafiki_worker:0.0.4',
  'model_class': 'TfSingleHiddenLayerVector',
  'name': 'TfSingleHiddenLayerVector',
  'task': 'FEATURE_VECTOR_CLASSIFICATION',
  'user_id': '71f6705c-ea09-4d98-98d1-25181dd34ea2'}]

uri = "https://github.com/sdragon007/sea_dataset/blob/master/sea_train.zip?raw=true"

In [4]:
#create a training job
client.create_train_job(
    app='concept_drift',
    task='FEATURE_VECTOR_CLASSIFICATION',
    train_dataset_uri='https://github.com/sdragon007/sea_dataset/blob/master/sea_train.zip?raw=true',
    test_dataset_uri='https://github.com/sdragon007/sea_dataset/blob/master/sea_test.zip?raw=true',
    budget_type='MODEL_TRIAL_COUNT',
    budget_amount=2
)

{'app': 'concept_drift',
 'app_version': 1,
 'id': 'd57162b0-cd2e-4258-82b9-e79e88bfff2e'}

In [7]:
#get train job status
client.get_train_job(app='concept_drift')

{'app': 'concept_drift',
 'app_version': 1,
 'budget_amount': 2,
 'budget_type': 'MODEL_TRIAL_COUNT',
 'datetime_completed': None,
 'datetime_started': 'Sun, 28 Oct 2018 12:02:24 GMT',
 'id': 'd57162b0-cd2e-4258-82b9-e79e88bfff2e',
 'status': 'RUNNING',
 'task': 'FEATURE_VECTOR_CLASSIFICATION',
 'test_dataset_uri': 'https://github.com/sdragon007/sea_dataset/blob/master/sea_test.zip?raw=true',
 'train_dataset_uri': 'https://github.com/sdragon007/sea_dataset/blob/master/sea_train.zip?raw=true',
 'workers': [{'datetime_started': 'Sun, 28 Oct 2018 12:02:24 GMT',
   'datetime_stopped': None,
   'model_name': 'SkDtVector',
   'replicas': 2,
   'service_id': 'bf1feac8-a53b-409c-8550-2a258be56620',
   'status': 'RUNNING'},
  {'datetime_started': 'Sun, 28 Oct 2018 12:02:24 GMT',
   'datetime_stopped': None,
   'model_name': 'SkSvmVector',
   'replicas': 2,
   'service_id': '58e10856-afe3-424f-8b15-01fc8af84d5a',
   'status': 'RUNNING'},
  {'datetime_started': 'Sun, 28 Oct 2018 12:02:24 GMT',
  

In [8]:
#get job metrics
client.get_best_trials_of_train_job(app='concept_drift')

[]

In [ ]:
#startinference job
client.create_inference_job(app='concept_drift')

In [ ]:
#stop inference job
client.stop_inference_job(app='concept_drift')